<a href="https://colab.research.google.com/github/larissavaladao/py6s_harmonize_sample/blob/main/2.cloudMask_deglint/curuai_acolite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import packages

In [44]:
#import packages used
import ee
import pandas as pd
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import math
import geemap
import os

In [45]:
#authenticate and initialize google earth engine (also necessary for geemap)
ee.Authenticate()
ee.Initialize(project='ee-curuai') #change project name here

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Curuai dataset

In [ ]:
#function to insert a property with date of the point - Gee date format
def insert_date (feat):
    return feat.set('system:time_start',ee.Date.parse('YYYY-MM-dd HH:mm:ss',feat.get('DATE')))

In [47]:
#apply date function - new system:time_start property
roi_points = ee.FeatureCollection('projects/ee-curuai/assets/curuai_points').map(insert_date)
print(roi_points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-55.455998192971315, -2.26245869249737]}, 'id': '000000000000000000ea', 'properties': {'DATE': '2014-11-30 00:00:00', 'ID': 234, 'LATITUDE': '-2.26246', 'LONGITUDE': '-55.456', 'MISSION': 'IV', 'system:time_start': {'type': 'Date', 'value': 1417305600000}}}


In [48]:
#create a polygon around the floodpalin area - roi
roi_poly = ee.FeatureCollection('projects/ee-curuai/assets/roi_poly_curuai')

# Import GEE images

## Define date ranges for filtering images

In [ ]:
def date_mission(mission,time):
  return ee.Date(roi_points.filter(ee.Filter.eq('MISSION',mission)).sort('system:time_start',time).first().get('system:time_start'))

In [49]:
mission = 'IX'
advance = 16

In [50]:
if mission == "I":
  col = 1
elif mission == "II":
  col = 2
elif mission == "III":
  col = 3
elif mission == "IV":
  col = 4
elif mission == "V":
  col = 5
elif mission == "VI":
  col = 6
elif mission == "VII":
  col = 7
elif mission == "VIII":
  col = 8
elif mission == "IX":
  col = 9
else:
  print('Mission value error')

In [51]:
#define initial and final date to filter the image collection based on the
#field points date
initial_date = date_mission(mission,True).advance(-advance, 'day')
end_date = date_mission(mission,False).advance(advance, 'day')

print('Data inicial de coleta: ',initial_date.format().getInfo())
print('Data final de coleta: ',end_date.format().getInfo())

Data inicial de coleta:  2017-08-27T00:00:00
Data final de coleta:  2017-10-05T00:00:00


## Import image collections and apply filters and cloud masks

###Landsat 7 BOA

importar coleção com correção Py6S

In [56]:
ld7_path = 'projects/ee-curuai/assets/Py6S/mission'+str(col)+'/LD7_mission'+str(col)
# print(ld7_path)
landsat7 = ee.ImageCollection(ld7_path)
landsat7

In [57]:
landsat7.aggregate_array('LANDSAT_PRODUCT_ID')

In [58]:
landsat7.first()

###Landsat 8 BOA

importar coleção com correção Py6S

In [60]:
ld8_path = 'projects/ee-curuai/assets/Py6S/mission'+str(col)+'/LD8_mission'+str(col)
landsat8 = ee.ImageCollection(ld8_path)
landsat8

In [61]:
landsat8.aggregate_array('LANDSAT_PRODUCT_ID')

In [62]:
landsat8.first()

###Sentinel 2 BOA

importar coleção com correção Py6S

In [63]:
s2_path = 'projects/ee-curuai/assets/Py6S/mission'+str(col)+'/sentinel2_mission'+str(col)
sentinel2 = ee.ImageCollection(s2_path)
sentinel2

In [64]:
sentinel2.aggregate_array('PRODUCT_ID')

In [65]:
sentinel2.first()

# Preprocessing

## Mask Clouds

### Cloud Masking functions

Sentinel 2 mask

Landsat mask

In [67]:
# Function to mask clouds using the quality band of Landsat 7 and 8.
def maskLD (image):
    qa = image.select('QA_PIXEL')
  # Check that the cloud bit is off.
  # See https://www.usgs.gov/media/files/landsat-8-9-olitirs-collection-2-level-1-data-format-control-book
    mask = qa.bitwiseAnd(1 << 3).eq(0)
    return image.updateMask(mask).copyProperties(image,['system:index','system:time_start',"CLOUD_COVER"]).set("PRODUCT_ID",image.get('LANDSAT_PRODUCT_ID'))

### Landsat 7

In [68]:
landsat7 = landsat7.map(maskLD)
landsat7

### Landsat 8

In [69]:
landsat8 = landsat8.map(maskLD)
landsat8

### Sentinel 2

In [ ]:
sentinel2_filter = sentinel2.filter(ee.Filter.inList('Orbit', sentinel2_acollite.aggregate_array('Orbit')))\
                    .filter(ee.Filter.inList('date_S2', sentinel2_acollite.aggregate_array('date_S2')))
sentinel2_filter

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [ ]:
sentinel2_combine = ee.ImageCollection(sentinel2_filter.map(combine_img(sentinel2_acollite)))\
          .map(maskS2_1C)\
          .map(maskS2)
sentinel2_combine

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


### visualização das imagens

Primeira imagem com menos nuvens da colação de iamgens disponiveis durante a missão

In [ ]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(roi_points,10)

Map.addLayer(ld8_combine.first(), {'bands':['b3','b2','b1'], min:0,max:0.00002}, str(ld8_combine.first().get('LANDSAT_PRODUCT_ID').getInfo()))
Map.addLayer(ld8_combine.first(), {'bands':['B4','B3','B2'], min:0,max:0.00002}, "TOA"+str(ld8_combine.first().get('LANDSAT_PRODUCT_ID').getInfo()))
Map.addLayer(ld7_combine.first(), {'bands':['b3','b2','b1'], min:0,max:0.00001}, str(ld7_combine.first().get('LANDSAT_PRODUCT_ID').getInfo()))
Map.addLayer(ld7_combine.first(), {'bands':['B3','B2','B1'], min:0,max:0.00002}, "TOA"+str(ld7_combine.first().get('LANDSAT_PRODUCT_ID').getInfo()))
Map.addLayer(sentinel2_combine.first(), {'bands':['b3','b2','b1'], min:0,max:0.00001}, str(sentinel2_combine.first().get('PRODUCT_ID').getInfo()))
Map.addLayer(sentinel2_combine.first(), {'bands':['B4','B3','B2'], min:0,max:0.00002}, "TOA"+str(sentinel2_combine.first().get('PRODUCT_ID').getInfo()))

Map.addLayer(roi_points, {'color':'darkred'}, 'Data Points');
# Map.addLayer(roi_poly, {'color':'darkred'}, 'Data polygon');
Map

Map(center=[-2.1837863104242503, -55.48976569658883], controls=(WidgetControl(options=['position', 'transparen…

## Padronize band names and projection

In [ ]:
#projeção
prj = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_228061_20161111').select(['B4']).projection()
prj

###Landsat 7

In [ ]:
#renomear bandas
name_bands = ['blue','green','red','nir','swir1','swir2']
ld7 = ld7_combine.map(lambda img: img.select(['b1','b2','b3','b4','b5','b6'])\
                        .rename(name_bands))

In [ ]:
print(ld7.first().select('red').projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32621', 'transform': [30, 0, 609900, 0, -30, -219540]}


### Landsat 8

In [ ]:
ld8 = ld8_combine.map(lambda img: img.select(['b1','b2','b3','b4','b5','b6'])\
                        .rename(name_bands))

In [ ]:
print(ld8.first().select('red').projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32621', 'transform': [30, 0, 676170, 0, -30, -219540]}


### Sentinel 2

In [ ]:
s2 = sentinel2_combine.map(lambda img: img.select(['b1','b2','b3','b4','b5','b6'])\
                        .rename(name_bands))

In [ ]:
print(s2.first().select('red').projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32721', 'transform': [10, 0, 609920, 0, -10, 9780460]}


## Transform into remote sensing  reflectance and sunglint correction

In [ ]:
#dividir imagem corrigida por pi
# Rrs_sat_ac = Rsat_ac / pi

#fazer deglint
#Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
#correção no artigo INPE CURUAI
def deglint (img):
    Rrs = img.divide(math.pi)
    deglint = Rrs.select(['blue','green','red','nir'])\
    .subtract(Rrs.select('swir1'))

    return (deglint.updateMask(deglint.select(['red']).gt(0))
    .copyProperties(img,['system:time_start','CLOUD_COVER',"system:index","LANDSAT_PRODUCT_ID","PRODUCT_ID","CLOUDY_PIXEL_PERCENTAGE"]))


### Landsat 7

In [ ]:
landsat7_rs = ld7.map(deglint).map(lambda img : img.set("SATELLITE_PRODUCT_ID",img.get("LANDSAT_PRODUCT_ID")))

landsat7_rs

### Landsat 8

In [ ]:
landsat8_rs = ld8.map(deglint).map(lambda img : img.set("SATELLITE_PRODUCT_ID",img.get("LANDSAT_PRODUCT_ID")))


### Sentinel 2

In [ ]:
sentinel2_rs = s2.map(deglint).map(lambda img : img.set({"SATELLITE_PRODUCT_ID":img.get("PRODUCT_ID"),
                                                          "CLOUD_COVER":img.get("CLOUDY_PIXEL_PERCENTAGE")}))

sentinel2_rs

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [ ]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(roi_points,10)
vis =  {'bands':['red','green','blue'],'min':0.01,'max':0.06}
Map.addLayer(landsat7_rs.sort('CLOUD_COVER').first(),vis, 'landsat7')
Map.addLayer(landsat8_rs.sort('CLOUD_COVER').first(),vis, 'landsat8')
Map.addLayer(sentinel2_rs.first(), vis, 'sentinel2')

Map.addLayer(roi_points, {'color':'darkred'}, 'Data Points');
# Map.addLayer(roi_poly, {'color':'darkred'}, 'Data polygon');
Map

Map(center=[-2.1837863104242503, -55.48976569658883], controls=(WidgetControl(options=['position', 'transparen…

# Sample data points pixel values

filter images that fall within a 16-day window period from each field point date

In [ ]:
def imgs_points(collection):
    def wrap(feat):
        date_point = ee.Date(feat.get('system:time_start'))
        data1 = date_point.advance(-16,'day')\
        .format('yyyy-MM-dd')

        data2 = date_point.advance(16,'day')\
        .format('yyyy-MM-dd')

        filtro = collection.filterDate(data1,data2)\
        .filterBounds(feat.geometry())\
        .map(lambda img: img.set({
            'dif_date_point':ee.Date(img.get('system:time_start')).difference(date_point, 'day')})\
            .copyProperties(feat,['ID']))


        return ee.ImageCollection(filtro.limit(10))

    return wrap

obtain statistics for the same location of the field point - with a 3 pixel window - and filter out if more than 4 pixels are masked   

In [ ]:
def sample_point(img):
  feat = roi_points.filter(ee.Filter.eq('ID',img.get('ID'))).first()
  geom = feat.geometry().buffer(45).bounds()

  mean = img.reduceRegion(geometry=geom,
                          scale=prj.nominalScale(),
                          crs=prj.crs(),
                          reducer=ee.Reducer.mean())
  median = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.median())
  minMax = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.minMax())
  count = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.count())
  return feat.set({
      "SATELLITE_PRODUCT_ID": img.get('SATELLITE_PRODUCT_ID'),
      'CLOUD_COVER':img.get('CLOUD_COVER'),
      'img_date':ee.Date(img.get('system:time_start')).format(),
      'dif_date_point': img.get('dif_date_point'),
      'blue_mean':mean.get('blue'),
      'green_mean':mean.get('green'),
      'red_mean':mean.get('red'),
      'nir_mean':mean.get('nir'),
      'blue_median':median.get('blue'),
      'green_meadin':median.get('green'),
      'red_median':median.get('red'),
      'nir_median':median.get('nir'),
      'blue_min':minMax.get('blue_min'),
      'green_min':minMax.get('green_min'),
      'red_min':minMax.get('red_min'),
      'nir_min':minMax.get('nir_min'),
      'blue_max':minMax.get('blue_max'),
      'green_max':minMax.get('green_max'),
      'red_max':minMax.get('red_max'),
      'nir_max':minMax.get('nir_max'),
      "count_pixel":count.get('red')
      })

filter_count = ee.Filter.gt('count_pixel',3)




### Landsat 7

In [ ]:
img_pointsLD7 = ee.ImageCollection(roi_points.map(imgs_points(landsat7_rs)).flatten().toList(2000))

In [ ]:
img_pointsLD7.size()


In [ ]:
img_pointsLD7.aggregate_count_distinct('ID')

In [ ]:
img_pointsLD7.limit(5)

In [ ]:
reduced_LD7 = ee.FeatureCollection(img_pointsLD7.map(sample_point)).filter(filter_count)

In [ ]:
reduced_LD7.limit(2)

In [ ]:
reduced_LD7.size()

In [ ]:
reduced_LD7.aggregate_count_distinct('ID')

## Landsat 8

In [ ]:
img_pointsLD8 = ee.ImageCollection(roi_points.map(imgs_points(landsat8_rs)).flatten().toList(2000))

In [ ]:
img_pointsLD8.size()


In [ ]:
img_pointsLD8.aggregate_count_distinct('ID')

In [ ]:
img_pointsLD8.limit(5)

In [ ]:
reduced_LD8 = ee.FeatureCollection(img_pointsLD8.map(sample_point)).filter(filter_count)

In [ ]:
reduced_LD8.size()


In [ ]:
reduced_LD8.aggregate_count_distinct('ID')

In [ ]:
reduced_LD8.limit(2)

##Sentinel 2

In [ ]:
img_pointsS2 = ee.ImageCollection(roi_points.map(imgs_points(sentinel2_rs)).flatten().toList(2000))

In [ ]:
img_pointsS2.size()

In [ ]:
img_pointsS2.aggregate_count_distinct('ID')

In [ ]:
reduced_S2 = ee.FeatureCollection(img_pointsS2.map(sample_point)).filter(filter_count)

In [ ]:
reduced_S2.size()


In [ ]:
reduced_S2.aggregate_count_distinct('ID')

In [ ]:
reduced_S2.limit(2)

## Visualize sampled image by defining field point ID

In [ ]:
Map = geemap.Map()
Map.centerObject(roi_points,10)
id = 302
vis_params={'bands':['red','green','blue'],'min':0,'max':0.05}
Map.addLayer(img_pointsLD7.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'landsat7')
Map.addLayer(img_pointsLD8.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'landsat8')
Map.addLayer(img_pointsS2.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'sentinel2')

Map.addLayer(roi_points.filter(ee.Filter.eq('ID',id)), {'color':'darkred'}, 'Data Points');
Map

Map(center=[-2.1837863104242503, -55.48976569658883], controls=(WidgetControl(options=['position', 'transparen…

# Export data as CSV file

In [ ]:
#Exportar tabelas
#LD7
geemap.ee_to_csv(reduced_LD7, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD7_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/V_LD7_ACOLLITE_data.csv


In [ ]:
geemap.ee_to_csv(reduced_LD8, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD8_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/V_LD8_ACOLLITE_data.csv


In [ ]:
geemap.ee_to_csv(reduced_S2, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD7_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/VI_S2_DOS_data.csv


CORREÇÂO ATM:


1.   ACOLITE https://github.com/acolite/acolite
2.   6S com imagem da Dinha e parametros do artigo... talvez seja necessaria mais uma correção para isso



